In [1]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd

In [415]:
#im just gonna try to use selenium i guess
from selenium import webdriver
import pandas as pd

In [652]:
def load_webpage():
    driver = webdriver.Chrome('/usr/bin/chromedriver')
    driver.get('https://nces.ed.gov/collegenavigator/')
    return driver

In [653]:
def get_search_options():
    el = driver.find_element_by_id('MapWrap')
    options = el.find_elements_by_tag_name('option')
    return options

In [657]:
def search_for_option(no_pref, option):
    no_pref.click() #deselect no preference
    option.click()
    bachelors_button = driver.find_element_by_id('ctl00_cphCollegeNavBody_ucSearchMain_chkBach')
    bachelors_button.click()
    search_button = driver.find_element_by_id('ctl00_cphCollegeNavBody_ucSearchMain_btnSearch')
    search_button.click()
    return

In [654]:
def get_student_pop():
    try:
        student_pop = driver.find_element_by_css_selector("#RightContent > div.dashboard > div > div:nth-child(2) > table > tbody > tr:nth-child(7) > td.srb + td")
        student_pop = re.findall('\d+', student_pop.text.replace(',',''))[0]
    except:
        student_pop = None
    return student_pop

In [692]:
def get_college_data_row():
    #right now I just grab the name and student population.
    #ill probably have individual funcs for grabbing each param later.
    college_data_row = []
    name = driver.find_element_by_css_selector(".headerlg").text
    student_pop = get_student_pop()
    college_data_row.extend([name, student_pop])
    print(college_data_row)
    return college_data_row

In [693]:
def get_data_on_page():
    data = []
    rows = driver.find_elements_by_css_selector(".resultsTable > tbody > tr")
    for ix, row in enumerate(rows):
        college_link = (driver
               .find_elements_by_css_selector(".resultsTable > tbody > tr")[ix]
               .find_element_by_css_selector("a"))
        college_link.click()
        row = get_college_data_row()
        data.append(row)
        driver.back()
        time.sleep(1)
    return data

In [694]:
def next_page(selected_page_num):
    css_addition = ' + a'*selected_page_num
    try:
        driver.find_element_by_css_selector('.colorful' + css_addition).click()
        return True
    except:
        return False

In [718]:
def get_data_for_all_pages():
    data = get_data_on_page()
    page_num = 1
    can_go_to_next_page = next_page(page_num)
    while can_go_to_next_page == True:
        data.extend(get_data_on_page())
        page_num += 1
        can_go_to_next_page = next_page(page_num)
    return data

In [719]:
def get_all_college_data(driver):
    option_ix = 1
    college_data = []
    more_options = True
    while more_options:
        time.sleep(.2)
        options = get_search_options()
        no_pref = options[0]
        try:
            option = options[option_ix]
            search_for_option(no_pref, option)
            college_data.extend(get_data_for_all_pages())
            driver.get('https://nces.ed.gov/collegenavigator/')
            option_ix += 1
        except:
            more_options = False
    return college_data

In [720]:
driver = load_webpage()
get_all_college_data(driver)

['Alabama A & M University', '6106']
['Alabama State University', '4413']
['Amridge University', '755']
['Athens State University', '3044']
['Auburn University', '30440']
['Auburn University at Montgomery', '5211']
['Birmingham-Southern College', '1268']
['Columbia Southern University', '21104']
['Faulkner University', '3178']
['Heritage Christian University', '78']
['Herzing University-Birmingham', '571']
['Huntingdon College', '1081']
['Huntsville Bible College', '128']
['Jacksonville State University', '8479']
['Judson College', '268']
['Miles College', '1550']
['Oakwood University', '1636']
['Remington College-Mobile Campus', '313']
['Samford University', '5619']
['Selma University', '317']
['South University-Montgomery', '390']
['Spring Hill College', '1379']
['Stillman College', '797']
['Strayer University-Alabama', '1234']
['Talladega College', '1212']
['The University of Alabama', '38390']
['Troy University', '16981']
['Tuskegee University', '3026']
['United States Sports Acade

['MiraCosta College', '13720']
['Modesto Junior College', '17908']
["Mount Saint Mary's University", '2965']
['Musicians Institute', '1105']
['National University', '16930']
['New York Film Academy', '1389']
['Newschool of Architecture and Design', '537']
['Northcentral University', '10535']
['Northwestern Polytechnic University', None]
['Notre Dame de Namur University', '1492']
['Occidental College', '1930']
['Otis College of Art and Design', '1144']
['Pacific College', '301']
['Pacific Oaks College', '1111']
['Pacific States University', '62']
['Pacific Union College', '1062']
['Palo Alto University', '1071']
['PCI College', '109']
['Pepperdine University', '7961']
['Pitzer College', '1072']
['Platt College-Anaheim', '349']
['Platt College-Los Angeles', '570']
['Platt College-Ontario', '510']
['Platt College-Riverside', '484']
['Platt College-San Diego', '145']
['Point Loma Nazarene University', '4592']
['Pomona College', '1573']
['Presbyterian Theological Seminary in America', '196'

['Eckerd College', '2006']
['Edward Waters College', '2906']
['Embry-Riddle Aeronautical University-Daytona Beach', '6613']
['Embry-Riddle Aeronautical University-Worldwide', '14168']
['Everglades University', '2262']
['Faith Theological Seminary and Christian College', None]
['Flagler College-St Augustine', '2608']
['Florida Agricultural and Mechanical University', '10021']
['Florida Atlantic University', '29772']
['Florida College', '508']
['Florida Gateway College', '3385']
['Florida Gulf Coast University', '15031']
['Florida Institute of Technology', '6271']
['Florida Institute of Technology-Online', '2310']
['Florida International University', '57942']
['Florida Memorial University', '1189']
['Florida National University-Main Campus', '4044']
['Florida Polytechnic University', '1425']
['Florida Southern College', '3154']
['Florida SouthWestern State College', '16556']
['Florida State College at Jacksonville', '23413']
['Florida State University', '41005']
['Florida Technical Colle

['Illinois State University', '20635']
['Illinois Wesleyan University', '1693']
['Judson University', '1256']
['Knox College', '1333']
['Lake Forest College', '1512']
['Lakeview College of Nursing', '168']
['Lewis University', '6440']
['Lincoln Christian University', '714']
['Lincoln College', '968']
['Loyola University Chicago', '17007']
['MacMurray College', '548']
['McKendree University', '2494']
['Methodist College', '692']
['Millikin University', '2094']
['Monmouth College', '914']
['Moody Bible Institute', '3086']
['National Louis University', '5992']
['National University of Health Sciences', '655']
['North Central College', '2928']
['North Park University', '2900']
['Northeastern Illinois University', '8103']
['Northern Illinois University', '17169']
['Northwestern College', '597']
['Northwestern University', '22127']
['Olivet Nazarene University', '4655']
['Principia College', '447']
['Quincy University', '1086']
['Rasmussen College-Illinois', '1729']
['Resurrection University

[['Alabama A & M University', '6106'],
 ['Alabama State University', '4413'],
 ['Amridge University', '755'],
 ['Athens State University', '3044'],
 ['Auburn University', '30440'],
 ['Auburn University at Montgomery', '5211'],
 ['Birmingham-Southern College', '1268'],
 ['Columbia Southern University', '21104'],
 ['Faulkner University', '3178'],
 ['Heritage Christian University', '78'],
 ['Herzing University-Birmingham', '571'],
 ['Huntingdon College', '1081'],
 ['Huntsville Bible College', '128'],
 ['Jacksonville State University', '8479'],
 ['Judson College', '268'],
 ['Miles College', '1550'],
 ['Oakwood University', '1636'],
 ['Remington College-Mobile Campus', '313'],
 ['Samford University', '5619'],
 ['Selma University', '317'],
 ['South University-Montgomery', '390'],
 ['Spring Hill College', '1379'],
 ['Stillman College', '797'],
 ['Strayer University-Alabama', '1234'],
 ['Talladega College', '1212'],
 ['The University of Alabama', '38390'],
 ['Troy University', '16981'],
 ['Tus

In [717]:
driver = load_webpage()
option_ix = 1
college_data = []
more_options = True
while more_options:
    time.sleep(.2)
    options = get_search_options()
    no_pref = options[0]
    try:
        option = options[option_ix]
        test = driver.find_element_by_id('MapWrap').find_elements_by_tag_name('option')
        print(test[option_ix].text)
        search_for_option(no_pref, option)
#         college_data.extend(get_data_for_all_pages())
        driver.back()
        option_ix += 1
        driver.refresh()
    except:
        more_options = False

Alabama
Alaska
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
District of Columbia
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky


In [711]:
test = driver.find_element_by_id('MapWrap').find_elements_by_tag_name('option')

In [714]:
options[2].text

'Alaska'